# THE WHAT?

Where shall agile go?

NB: Make sure to update taxonomy at the end!

In [1]:
file = '../../data/papersummaries_cleaned.csv'

In [2]:
import pandas as pd
import re

In [175]:
df = pd.read_csv(file)

NB: Because 'validate' is also used for the meta studies (validation of some procedure), we will need to remove 'bad tags' later

In [176]:
def helper(regex):
    res = [x for x in df.PaperSummary if re.search(regex, x)]
    return (len(res), res)

def give(loc):
    return df.PaperSummary[loc]

def match(loc):
    summary = give(loc)
    return (summary, assign_all_what(summary))

### The Rules

In [177]:
def assign_tag(level_1, level_2, level_3, summary):
    tags = []
    for tag in level_2:
        if any([re.search(keyword, summary) for keyword in level_3[tag]]):
            tags.append(level_1+tag)
    return tags

In [601]:
def assign_phase(summary):
    level_1 = ':phase'
    
    level_2 = [':elicitation', ':analysis', ':specification', ':evaluation', ':management']
    
    level_3 = {':elicitation':
                   ['[Ee]licit', '[Dd]iscover']
               , 
               ':analysis':
                   ['[Aa]naly[sz]ing\srequirements', 'requirements\sanalysis']
               , 
               ':specification':
                   ['[Rr]equirements?\sspec', 'specify']
               ,
               ':evaluation':
                   ['[Vv]erif', '[Vv]alidat', '[Tt]esting.*?(?!hypothes)']
               ,
               ':management':
                   ['[Mm]aint[ae]', '\s[Aa]dapt', '\s[Rr]eus[ei]', 'continuous', '\schange',
                    '[Rr]ecover', '\s[Ee]volution|\sevolv[ei]', '[Mm]onitor', '[Rr]untime'
                   ]
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [610]:
def assign_quality(summary):
    level_1 = ':information:quality'
    
    level_2 = ['_compatibility', '_maintainability', '_performance', 
               '_portability', '_reliability', '_safety', '_security', '_usability', '_all']
    
    level_3 = {'_compatibility':   ['[Cc]ompatib'], 
               '_maintainability': [], 
               '_performance':     ['[Pp]erformance'],
               '_portability': ['[Pp]ortab'],
               '_reliability': ['reliab', 'dependab'],
               '_safety':      ['[Ss]afety\s'],
               '_security':    ['[Ss]ecur', '[Pp]rivac'],
               '_usability':   ['[Uu]sability'],
               '_all': ['[Qq]uality\s[Rr]eq', 'non-functional']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [727]:
def assign_information(summary):
    level_1 = ':information'
    
    level_2 = [':goals', ':systembehavior', ':functional']
    
    level_3 = {':goals': ['stakeholder\sgoals'], 
               ':systembehavior': ['behaviou?ral\sreq'],
               ':functional': ['\sfunctional\sreq']
               }
    
    return assign_tag(level_1, level_2, level_3, summary) + assign_quality(summary)

In [728]:
def assign_documentation(summary):
    level_1 = ':documentation'
    
    level_2 = [':diagrams', ':businessmodels', ':goalmodels', ':statemachines', ':featuremodels',
               ':naturallanguage', ':prototypes', ':usecases', ':userstories']
    
    level_3 = {':diagrams':       ['[Dd]iagram', 'UML'], 
               ':businessmodels': ['business\s(process\s)?models?'], 
               ':goalmodels':     ['goal\smodel', 'i\*'],
               ':featuremodels':  ['feature\smodel'],
               ':statemachines':  ['state\smachine'],
               ':naturallanguage':['natural\slanguage', '\swords', '[- ]text'],
               ':prototypes':     ['[Pp]rototyp'],
               ':usecases':
                   ['use\scase']
               ,
               ':userstories':
                   ['user\sstories']
               ,
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [729]:
def assign_challenge_content(summary):
    level_1 = ':challenge:content'
    
    level_2 = ['_unambiguousness', '_consistency', 
               '_feasibility', '_traceability', 
               '_completeness', '_understandability',
               '_all'
              ]
    
    level_3 = {'_unambiguousness':
                   ['ambigu', 'clarif', 'vague']
               ,
               '_consistency':
                   ['inconsisten']
               ,
               '_feasibility':
                   []
               ,
               '_traceability':
                   ['\strac']
               ,
               '_completeness':
                   ['completeness', 'incomplete', 'insufficient', 'missing\srequire']
               ,
               '_understandability':
                   ['understandability', 'comprehens(?:ion|ibility)'],
               '_all':
                   ['requirements?\sdefect', 'requirements?\serror']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [730]:
def assign_challenge_people(summary):
    level_1 = ':challenge:people'
    
    level_2 = ['_communication', '_collaboration', '_subjectivity', '_skills'
              ]
    
    level_3 = {'_communication':
                   ['[Cc]ommunica', 'misunderst', 'feedback']
               ,
               '_collaboration':
                   ['[Cc]ollabor']
               ,
               '_subjectivity':
                   ['[Pp]erce(?:pti|iv)', '[Bb]elie[fv]', 'subjectiv']
               ,
               '_skills':
                   ['educat', '(?<!machine\s)learning', 'skills', 'teaching', 'creativ', '\sjobs?\s']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [731]:
def assign_challenge_process(summary):
    level_1 = ':challenge:process'
    
    level_2 = ['_automation', '_formalization', '_standardization'
               , '_prioritization', '_visualization', '_deciding', '_improving'
              ]
    
    level_3 = {'_automation':      ['[Aa]utomat(?:ic|ion|ing|es|e)'],
               '_formalization':   ['[Ff]ormaliz'],
               '_prioritization':  ['[Pp]riorit'],
               '_standardization': ['[Ss]tandardiz', '[Tt]emplate', '[Rr]eus[ea]', '[Bb]lueprint'],
               '_visualization':   ['[Vv]isual'],
               '_deciding':        ['[Dd]ecisions[- ]making', '[Dd]ecide(?!d)', '[Dd]ecisions?\sprobl'],
               '_improving':       ['improve'] 
               # i am including this to see whether there's bias due to positively primed words
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [732]:
def assign_challenge_context(summary):
    level_1 = ':challenge:context'
    
    level_2 = ['_uncertainty', '_regulation', ]
    
    level_3 = {'_uncertainty':   ['uncertain'],
               '_regulation':    ['regulat[io]', 'jurisdict', 'legal'],
               '_ethics':        ['ethical']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [733]:
def assign_domain_class(summary):
    level_1 = ':domain:systemclass'
    
    level_2 = ['_embedded', '_safetycritical', '_adaptive', '_complex']
    
    level_3 = {'_embedded': ['[Ee]mbedded\s[Ss]ystem'],
               '_safetycritical': ['[Aa]utomotive', '[Ss]ubsea\s[Ee]quipment'],
               '_adaptive': ['[Aa]daptive\s[Ss]ystem'],
               '_complex': ['[Cc]omplex.*?[Ss]ystem'],
               '_opensource': ['[Oo]pen\s[Ss]ource']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

def assign_domain_sector(summary):
    level_1 = ':domain:sector'
    
    level_2 = ['_automotive', '_subsea', '_mobile', '_media', '_it', '_health']
    
    level_3 = {'_automotive': ['[Aa]utomotive'],
               '_subsea': ['[Ss]ubsea'],
               '_mobile': ['[Mm]obile'],
               '_media': ['entertainment', '\sgames?'],
               '_it': ['\s[Ww]eb\s', 'IT\scompany'],
               '_health': ['health', '\sinjur(?:y|ies)']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

def assign_domain_organization(summary):
    level_1 = ':domain:organization'
    
    level_2 = ['_distributed', '_agile', ]
    
    level_3 = {'_distributed': ['\sdistributed\s'],
               '_agile':       ['\sagile\s'],
               '_outsourced':  ['outsource']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [734]:
def assign_region(summary):
    level_1 = ':region'
    
    level_2 = [':continent_europe', ':continent_asia', ':continent_namerica', 
               ':continent_samerica', ':continent_africa', ':continent_australia',
               ':country_finland', ':country_china'
              ]
    
    level_3 = {x:[] for x in level_2
               }
    
    level_3[':country_finland'] = ['Finland']
    level_3[':country_russia'] = ['Russia']
    
    return assign_tag(level_1, level_2, level_3, summary)

In [735]:
def assign_general(summary):
    level_1 = ':general'
    
    level_2 = [':research'
              ]
    
    level_3 = {':research': ['literature.*?research', 'threats\sto\svalidity', ],
               
               }
    
    return assign_tag(level_1, level_2, level_3, summary)
    

In [736]:
def assign_all_what(summary):
    tags = ['what'+ x for x in 
            (
                assign_phase(summary) # add further things here
                + assign_information(summary)
                + assign_documentation(summary)
                + assign_challenge_content(summary)
                + assign_challenge_people(summary)
                + assign_challenge_process(summary)
                + assign_challenge_context(summary)
                + assign_domain_class(summary)
                + assign_domain_sector(summary)
                + assign_domain_organization(summary)
                + assign_general(summary)
            )]
    return tags

In [737]:
match(157)

('A modelling language for modelling the behavioural requirements of a software product line in order to ease the task of adding new features to a set of existing requirements.',
 ['what:information:systembehavior'])

In [738]:
helper('subjectiv')

(1,
 ['A method for assessing the completeness of specification documents in order to avoid subjectiveness in such completeness assessment'])

In [739]:
#[assign_all_what(x) for x in df.PaperSummary]

In [740]:
df['what'] = [assign_all_what(x) for x in df.PaperSummary]

In [741]:
df.head()

,PaperID,PaperSummary,what
0,1,A method for automatically recovering software...,"[what:phase:management, what:challenge:content..."
1,2,A set of two techniques for improving the qual...,"[what:challenge:content_traceability, what:cha..."
2,3,A case study on evaluating a given technique f...,[what:challenge:process_prioritization]
3,4,An experience report on the development of a m...,"[what:phase:evaluation, what:challenge:process..."
4,5,A document-driven study on the relevancy of cl...,[what:phase:specification]


Summaries without tags?

In [742]:
len([(x[0], x[1][1], x[1][2]) for x in df.iterrows() if len(x[1][2]) < 1])

58

In [743]:
[(x[0], x[1][1], x[1][2]) for x in df.iterrows() if len(x[1][2]) < 1][30:]

[(204,
  'A case study for understanding the root causes of failed software projects in order to identify risk areas and provide practical guidance to practitioners',
  []),
 (205,
  'A case study on requirements brainstorming sessions considering knowledge of the domain as the factor to be measured in order to assess the effectiveness of idea generation',
  []),
 (206,
  'A method for identifying unstated assumptions in composite service-oriented applications in order to decrease the risk of unexpected behavior',
  []),
 (208,
  'A method for predicting integration bugs based on network measures calculated over requirements dependency networks',
  []),
 (211,
  'An experience report on the development of three generations of a product in order to assess the positive effects of well-written and well-reviewed requirements on the quality of the final product',
  []),
 (219,
  'An experience report on the requirements engineering process for a national Police Force Crime Records Managemen

Summaries with multiple tags? (to check whether they're okay)

In [744]:
#[(x[0], x[1][1], x[1][2]) for x in df.iterrows() if len(x[1][2]) > 1]

In [745]:
longform_tags = pd.DataFrame(columns=['PaperID', 'Tag'])
for row in df.iterrows():
    longform_tags = longform_tags.append(
        pd.DataFrame({'PaperID':list(len(row[1][2])*[row[1][0]]), 'Tag':row[1][2]}))
    #longform_tags = longform_tags.append(
        #pd.DataFrame({'PaperID':list(len(row[1][3])*[row[1][0]]), 'Tag':row[1][3]}))
longform_tags = longform_tags.reset_index().drop('index', axis=1)
longform_tags.head()

,PaperID,Tag
0,1,what:phase:management
1,1,what:challenge:content_traceability
2,1,what:challenge:process_automation
3,2,what:challenge:content_traceability
4,2,what:challenge:context_regulation


In [746]:
longform_tags.shape

(671, 2)

In [747]:
tags_four_levels = list(zip(*[x.split(':') for x in longform_tags.Tag]))
longform_tags['level_1'] = tags_four_levels[0]
longform_tags['level_2'] = tags_four_levels[1]
longform_tags['level_3'] = [x.split('_')[0] for x in tags_four_levels[2]]
longform_tags['level_4'] = [x.split('_')[-1] if (len(x.split('_')) > 1) 
                                                    else '' for x in tags_four_levels[2]]
longform_tags.head()

,PaperID,Tag,level_1,level_2,level_3,level_4
0,1,what:phase:management,what,phase,management,
1,1,what:challenge:content_traceability,what,challenge,content,traceability
2,1,what:challenge:process_automation,what,challenge,process,automation
3,2,what:challenge:content_traceability,what,challenge,content,traceability
4,2,what:challenge:context_regulation,what,challenge,context,regulation


NB: some papers appear multiple times in the counts (here: twice at max) since some papers receive multiple tags.

In [748]:
longform_tags.groupby(['level_1', 'level_2', 'level_3', 'level_4']).count()[['PaperID']]

PaperID
level_1 level_2       level_3         level_4                   
what    challenge     content         all                      4
                                      completeness            12
                                      consistency              8
                                      traceability            41
                                      unambiguousness         18
                                      understandability        5
                      context         regulation              20
                                      uncertainty             11
                      people          collaboration            9
                                      communication           19
                                      skills                  14
                                      subjectivity             9
                      process         automation              47
                                      deciding                 2
                                      formalization            1
                                      improving               34
                                      prioritization          15
                                      standardization         17
                                      visualization           12
        documentation businessmodels                           1
                      diagrams                                 7
                      featuremodels                            6
                      goalmodels                               7
                      naturallanguage                         39
                      prototypes                               1
                      statemachines                            1
                      usecases                                10
                      userstories                              4
        domain        organization    agile                   11
                                      distributed              2
                      sector          automotive               4
                                      health                   3
                                      it                       4
                                      media                    6
                                      mobile                   4
                                      subsea                   1
                      systemclass     adaptive                 6
                                      complex                  1
                                      embedded                 2
                                      safetycritical           5
        general       research                                 5
        information   functional                               7
                      goals                                    2
                      quality         all                     17
                                      performance              8
                                      reliability              2
                                      safety                   6
                                      security                26
                                      usability                2
                      systembehavior                           3
        phase         analysis                                 6
                      elicitation                             43
                      evaluation                              28
                      management                              66
                      specification                           27

The End.